# Exercício Aula 06 - Gerando Redes Aleatórias e Small World

Author: Gabriel Van Loon

Prof.:  Francisco Aparecido Rodrigues

Universidade de São Paulo, São Carlos, Brasil.


## Definindo as Funções e Bibliotecas utilizadas

In [1]:
import collections
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from scipy.stats import pearsonr, spearmanr

from networkx.algorithms import community
from networkx.algorithms.community import greedy_modularity_communities
from community import community_louvain

In [43]:
def degree_distribution(GER):
    vk = dict(GER.degree())
    vk = list(vk.values()) # we get only the degree values
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

def momment_of_degree_distribution(G,m):
    k,Pk = degree_distribution(G)
    M = sum((k**m)*Pk)
    return M

def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H


def modularity(G, c):
    """
    Calcula a modularidade da partição realizada na rede G nas c
    comunidades. Outra opção é utilizar a função do networkx
    networkx.algorithms.community.quality.modularity(G,c)
    """
    A = nx.adjacency_matrix(G)
    N = len(G)
    M = G.number_of_edges()
    Q = 0
    for i in np.arange(0,N):
        ki = len(list(G.neighbors(i)))
        for j in np.arange(0,N):
            if(c[i]==c[j]):
                kj = len(list(G.neighbors(j)))
                Q = Q + A[i,j]-(ki*kj)/(2*M)
    Q = Q/(2*M)
    return Q

## 1) Gere um grafo aleatório com N=1000 e p = 0.1. Qual o valor do grau médio, segundo momento do grau e coeficiente de aglormeração médio (average clustering coefficient)?

In [62]:
N, p = (1000, 0.1)
G = nx.gnp_random_graph(N, p, seed=None, directed=False)

In [63]:
avg_deg = 2*G.number_of_edges()/N
moment  = momment_of_degree_distribution(G,2)
clustering = nx.average_clustering(G)

print("Average Degree:", avg_deg)
print("Second Moment", moment)
print("Average clustering:", clustering)

Average Degree: 99.348
Second Moment 9961.887999999999
Average clustering: 0.09969861159211599


## 2) Gere um small-world com N=1000, grau médio igual 10 e p = 0.1. Qual o valor do grau médio, segundo momento do grau e coeficiente de aglormeração médio (average clustering coefficient)?

In [117]:
N, k, p = (1000, 10, 0.1)
G = nx.watts_strogatz_graph(N, k, p, seed=None)

In [118]:
avg_deg = 2*G.number_of_edges()/N
moment  = momment_of_degree_distribution(G,2)
clustering = nx.average_clustering(G)

print("Average Degree:", avg_deg)
print("Second Moment", moment)
print("Average clustering:", clustering)

Average Degree: 10.0
Second Moment 100.904
Average clustering: 0.49868802308802385


## 3 ) Considere uma rede aleatória (Erdos-Renyi) com N=1000 vértices. Qual o valor da entropia de Shannon do grau para 〈𝑘〉= 5, 〈𝑘〉=10,〈𝑘〉=50

In [47]:
N, p = (1000, 0.005) # <k> = 5
G1 = nx.gnp_random_graph(N, p, seed=None, directed=False)

N, p = (1000, 0.01) # <k> = 10
G2 = nx.gnp_random_graph(N, p, seed=None, directed=False)

N, p = (1000, 0.05) # <k> = 50
G3 = nx.gnp_random_graph(N, p, seed=None, directed=False)

# o <k> é o primeiro momento, então eu preciso forçar o momento a ser 5?
# print(momment_of_degree_distribution(G,1))

In [48]:
print("Shannon Entropy = ", "%3.4f"%shannon_entropy(G1))
print("Shannon Entropy = ", "%3.4f"%shannon_entropy(G2))
print("Shannon Entropy = ", "%3.4f"%shannon_entropy(G3))

Shannon Entropy =  3.1078
Shannon Entropy =  3.6159
Shannon Entropy =  4.7820


## 4) Para o modelo small-world, calcule o valor da menor distância média (average shortest path) para p=0; p=0.01; p=0.05 e p=0.1. Considere grau médio igual a 4 e N = 100.

In [119]:
AVG1 = []
AVG2 = []
AVG3 = []
AVG4 = []

for i in range(30):
    N, k, p = (100, 4, 0)
    G1 = nx.connected_watts_strogatz_graph(N, k, p, seed=None)

    N, k, p = (100, 4, 0.01)
    G2 = nx.connected_watts_strogatz_graph(N, k, p, seed=None)

    N, k, p = (100, 4, 0.05)
    G3 = nx.connected_watts_strogatz_graph(N, k, p, seed=None)

    N, k, p = (100, 4, 0.1)
    G4 = nx.connected_watts_strogatz_graph(N, k, p, seed=None)
    
    AVG1.append(nx.average_shortest_path_length(G1))
    AVG2.append(nx.average_shortest_path_length(G2))
    AVG3.append(nx.average_shortest_path_length(G3))
    AVG4.append(nx.average_shortest_path_length(G4))

In [120]:
    print("Avg. shortest path len (0.00)", np.average(AVG1))
    print("Avg. shortest path len (0.01)", np.average(AVG2))
    print("Avg. shortest path len (0.05)", np.average(AVG3))
    print("Avg. shortest path len (0.10)", np.average(AVG4))

Avg. shortest path len (0.00) 12.878787878787879
Avg. shortest path len (0.01) 10.034329966329967
Avg. shortest path len (0.05) 6.258309764309765
Avg. shortest path len (0.10) 5.139508417508417


## 5) Considere o modelo de Erdos-Renyi. Gere redes com grau médio igual a 5, 10 e 50 e N=1000. Qual o valor da assortatividade?

In [102]:
ASS1 = []
ASS2 = []
ASS3 = []

for i in range(30):
    N, p = (1000, 0.005) # <k> = 5
    G1 = nx.gnp_random_graph(N, p, seed=None, directed=False)

    N, p = (1000, 0.01) # <k> = 10
    G2 = nx.gnp_random_graph(N, p, seed=None, directed=False)

    N, p = (1000, 0.05) # <k> = 50
    G3 = nx.gnp_random_graph(N, p, seed=None, directed=False)

    ASS1.append(nx.degree_assortativity_coefficient(G1))
    ASS2.append(nx.degree_assortativity_coefficient(G2))
    ASS3.append(nx.degree_assortativity_coefficient(G3))

In [103]:
print("Average Degree:", 2*G1.number_of_edges()/N)
print("Assortativity (k=5)", np.average(ASS1))
print("Average Degree:", 2*G2.number_of_edges()/N)
print("Assortativity (k=10)", np.average(ASS2))
print("Average Degree:", 2*G3.number_of_edges()/N)
print("Assortativity (k=50)", np.average(ASS3))

Average Degree: 5.058
Assortativity (k=5) -0.005378833808663564
Average Degree: 10.162
Assortativity (k=10) -0.003440663938922668
Average Degree: 50.06
Assortativity (k=50) -0.001366540466317087


## 6) Considere o modelo small-world. Gere redes com grau médio 10 e N=1000. Qual o valor da assortatividade para p=0.01; 0.05 e 0.2?

In [121]:
ASS1 = []
ASS2 = []
ASS3 = []

for i in range(30):
    N, k, p = (1000, 10, 0.01)
    G1 = nx.watts_strogatz_graph(N, k, p, seed=None)

    N, k, p = (1000, 10, 0.05)
    G2 = nx.watts_strogatz_graph(N, k, p, seed=None)

    N, k, p = (1000, 10, 0.1)
    G3 = nx.watts_strogatz_graph(N, k, p, seed=None)

    ASS1.append(nx.degree_assortativity_coefficient(G1))
    ASS2.append(nx.degree_assortativity_coefficient(G2))
    ASS3.append(nx.degree_assortativity_coefficient(G3))

In [122]:
print("Assortativity (p=0.01)", np.average(ASS1))
print("Assortativity (p=0.05)", np.average(ASS2))
print("Assortativity (p=0.10)", np.average(ASS3))

Assortativity (p=0.01) 0.0015758464283129666
Assortativity (p=0.05) -0.004606497818592103
Assortativity (p=0.10) -0.007041437816310548
